In [0]:
# SAS token
sas_token = "sp=racwdli&st=2025-09-03T05:45:03Z&se=2025-09-03T14:00:03Z&spr=https&sv=2024-11-04&sr=c&sig=lC3dWqTbd47C%2FkbpAwPtRsHX1PV73R0Sbq3Vi6CEtQw%3D"

storage_account_name = "rgassmartgear"
container_name = "raw"
file_name = "smartgear_sales.csv"

# Configure Hadoop to use SAS
spark.conf.set(
  f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
  sas_token
)

# Now build the path (without token in URL this time)
csv_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{file_name}"

# Read CSV
df = spark.read.option("header", "true").csv(csv_path)
df.show(5)


+-------+----------+------+-------+----------+--------+---------+
|OrderID| OrderDate|Region|StoreID|   Product|Quantity|UnitPrice|
+-------+----------+------+-------+----------+--------+---------+
|   1001|2025-03-14|  East|    115|Headphones|       3|    81.65|
|   1002|2025-02-19|  West|    110|Smartwatch|       3|    242.5|
|   1003|2025-03-17|  West|    113|   Printer|       2|   152.59|
|   1004|2025-01-05|  West|    118|    Camera|       4|    463.4|
|   1005|2025-01-07|  West|    110|    Tablet|       3|   370.01|
+-------+----------+------+-------+----------+--------+---------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import sum as _sum

In [0]:
df = df.withColumn("Quantity", df["Quantity"].cast("int")) #typecasting

In [0]:
# Aggregate total quantity by region
region_qty_df = df.groupBy("Region").agg(_sum("Quantity").alias("Total_Quantity"))

region_qty_df.show()

+------+--------------+
|Region|Total_Quantity|
+------+--------------+
| South|           740|
|  East|           719|
|  West|           694|
| North|           848|
+------+--------------+



In [0]:
# Apply SAS for both blob + dfs endpoints
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.dfs.core.windows.net",
    sas_token
)


# Output path
parquet_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/region_qty"

# Force overwrite + Parquet
(region_qty_df
    .coalesce(1)                 # optional → single file
    .write.mode("overwrite")     # clean old folder
    .option("overwriteSchema", "true")  # avoid schema mismatch
    .format("parquet")           # force Parquet (not Delta)
    .save(parquet_path))

print("✅ region_qty written successfully to Blob Storage as Parquet")

✅ region_qty written successfully to Blob Storage as Parquet
